In [46]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import numpy as np
import pandas as pd
import QuantLib as ql
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from pandas.tseries.offsets import BDay

import scipy.interpolate
from scipy.optimize import newton, fmin_slsqp

from CurveDataFetcher import CurveDataFetcher
from CurveInterpolator import CurveInterpolator
from CurveBuilder import get_spot_rates_fitter, reprice_bonds
from models.calibrate import (
    calibrate_ns_ols,
    calibrate_nss_ols,
    calibrate_bc_ols,
    calibrate_bc_augmented_ols,
    calibrate_diebold_li_ols,
    calibrate_mles_ols,
    calibrate_pca_yield_curve,
    calibrate_smith_wilson_ols,
)
from utils.utils import pydatetime_to_quantlib_date, quantlib_date_to_pydatetime
from models.nss import ZeroCouponCurve
from utils.QL_BondPricer import QL_BondPricer
from utils.RL_BondPricer import RL_BondPricer

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, no_logs_plz=True)

In [48]:
as_of_date = datetime(2024, 8, 28)
quote_type = "eod"

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True, 
    use_github=True, 
    include_off_the_run_number=True,
    market_cols_to_return=[f"{quote_type}_price", f"{quote_type}_yield"],
    calc_free_float=True,
)

# remove OTRs and first off the runs due to liquidity premium
filtered_curve_set_df = curve_set_df[(curve_set_df["rank"] != 0) & (curve_set_df["rank"] != 1)]

# remove TBills
filtered_curve_set_df = filtered_curve_set_df[
    filtered_curve_set_df["security_type"] != "Bill"
]

# remove low free float bonds (< $8bn)
filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["free_float"] > 8000] 

# filter bonds very close to maturity
filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["time_to_maturity"] > 0.25] 

# remove CTDs (this is a TODO)
# filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["is_ctd"] == False] 

filtered_curve_set_df

Using SOMA Holdings Data As of 2024-08-21
Using STRIPping Data As of 2024-07-31 00:00:00


,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,label,...,parValue,percentOutstanding,changeFromPriorWeek,changeFromPriorYear,outstanding_amt,portion_unstripped_amt,portion_stripped_amt,reconstituted_amt,free_float,rank
52,91282CFX4,Note,2022-11-21,2022-11-30,2024-11-30,0.257534,4.500,NaN,False,"Nov 24s, 2-Year",...,4.566032e+09,0.098063,0.0,0.000000e+00,4.656210e+10,46551301.7,1.080000e+07,0.0,41985.2693,20.0
53,91282CGD7,Note,2022-12-27,2023-01-03,2024-12-31,0.342466,4.250,NaN,False,"Dec 24s, 2-Year",...,0.000000e+00,NaN,NaN,NaN,4.198899e+10,41988991.2,0.000000e+00,0.0,41988.9912,19.0
54,91282CGG0,Note,2023-01-24,2023-01-31,2025-01-31,0.427397,4.125,NaN,False,"Jan 25s, 2-Year",...,0.000000e+00,NaN,NaN,NaN,4.199199e+10,41991993.5,0.000000e+00,0.0,41991.9935,18.0
55,91282CGN5,Note,2023-02-21,2023-02-28,2025-02-28,0.504110,4.625,NaN,False,"Feb 25s, 2-Year",...,9.542573e+09,0.185187,0.0,0.000000e+00,5.152952e+10,51529515.1,0.000000e+00,0.0,41986.9417,17.0
56,91282CGU9,Note,2023-03-27,2023-03-31,2025-03-31,0.589041,3.875,NaN,False,"Mar 25s, 2-Year",...,0.000000e+00,NaN,NaN,NaN,4.199558e+10,41993978.2,1.600000e+06,0.0,41993.9782,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,912810TN8,Bond,2023-04-13,2023-04-17,2053-02-15,28.487671,3.625,NaN,False,"Feb 53s, 30-Year",...,9.367420e+09,0.141965,0.0,1.000000e+07,6.598403e+10,50170937.7,1.581309e+10,1534040.0,40803.5178,6.0
386,912810TR9,Bond,2023-07-13,2023-07-17,2053-05-15,28.731507,3.625,NaN,False,"May 53s, 30-Year",...,5.723292e+09,0.091437,0.0,2.000000e+06,6.259307e+10,39104686.8,2.348839e+10,2265540.0,33381.3949,5.0
387,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,28.983562,4.125,NaN,False,"Aug 53s, 30-Year",...,8.606596e+09,0.120247,0.0,2.000000e+06,7.157430e+10,60802584.0,1.077172e+10,208800.0,52195.9878,4.0
388,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.235616,4.750,NaN,False,"Nov 53s, 30-Year",...,4.567153e+08,0.006874,0.0,4.567153e+08,6.644364e+10,57766113.9,8.677531e+09,1851960.0,57309.3986,3.0


In [50]:
# t2 = np.linspace(1/365, 30, 1000)

# fitted_zero_curve_func_ns, status_ns = calibrate_ns_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )
# assert status_ns

# fitted_zero_curve_func_nss, status_nss, lstsq_res_nss = calibrate_nss_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     tau0=(1, 3.5)
# )
# assert status_nss

# print(status_nss)
# print(lstsq_res_nss)

# fitted_zero_curve_func_dl, status_dl = calibrate_diebold_li_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )

# fitted_zero_curve_func_bc, status_bc = calibrate_bc_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# fitted_zero_curve_func_aug_bc, status_aug_bc = calibrate_bc_augmented_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# filtered_curve_set_mles_func, status_mles = calibrate_mles_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     overnight_rate=5.31,
#     N=6
# )


fitted_zero_curve_dict = get_spot_rates_fitter(
    curve_set_df=filtered_curve_set_df,
    as_of_date=as_of_date,
    on_rate=5.35,
    ql_fitting_methods=["ql_f_nss"],
    ql_zero_curve_interp_method="ql_z_interp_monot_cubic",
    daily_interpolation=True,
)

In [53]:
rv_df = reprice_bonds(as_of_date=as_of_date, ql_zero_curve=fitted_zero_curve_dict["ql_f_nss"]["ql_zero_curve"], curve_set_df=curve_set_df)
rv_df

,cusip,label,issue_date,maturity_date,time_to_maturity,high_investment_rate,int_rate,rank,outstanding,soma_holdings,stripping_amount,free_float,eod_yield,eod_price,accured,repriced_npv,repriced_ytm,price_spread,ytm_spread
0,912797LA3,"Sep 24s, 17-Week",2024-08-06,2024-09-03,0.016438,5.381000,NaN,16.0,0.000000e+00,6.784389e+08,0.000000e+00,-678.4389,5.420322,99.927361,0.000000,99.927462,5.382887,-0.000101,3.743474
1,912797LG0,"Sep 24s, 17-Week",2024-08-13,2024-09-10,0.035616,5.381000,NaN,15.0,0.000000e+00,6.873750e+08,0.000000e+00,-687.3750,5.433639,99.825333,0.000000,99.826844,5.356290,-0.001511,7.734910
2,912797LH8,"Sep 24s, 17-Week",2024-08-20,2024-09-17,0.054795,5.355000,NaN,14.0,0.000000e+00,7.210193e+08,0.000000e+00,-721.0193,5.457497,99.722389,0.000000,99.727299,5.330039,-0.004910,12.745823
3,912797LJ4,"Sep 24s, 17-Week",2024-08-27,2024-09-24,0.073973,5.335000,NaN,13.0,0.000000e+00,4.817243e+08,0.000000e+00,-481.7243,5.428712,99.622278,0.000000,99.628804,5.304132,-0.006526,12.458060
4,912797LK1,"Oct 24s, 17-Week",2024-08-06,2024-10-01,0.093151,5.346000,NaN,12.0,0.000000e+00,5.076268e+08,0.000000e+00,-507.6268,5.338675,99.526083,0.000000,99.531338,5.278564,-0.005255,6.011150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,912810TT5,"Aug 53s, 30-Year",2023-10-16,2053-08-15,28.983562,NaN,4.125,4.0,7.157430e+10,8.606596e+09,1.077172e+10,52195.9878,4.152796,99.531250,0.156929,97.843889,4.255298,1.844291,-10.250171
388,912810TV0,"Nov 53s, 30-Year",2024-01-16,2053-11-15,29.235616,NaN,4.750,3.0,6.644364e+10,4.567153e+08,8.677531e+09,57309.3986,4.134278,110.375000,1.368207,108.335474,4.249555,3.407732,-11.527704
389,912810TX6,"Feb 54s, 30-Year",2024-04-15,2054-02-15,29.487671,NaN,4.250,2.0,7.119879e+10,2.211754e+09,1.127392e+10,57713.1192,4.146389,101.750000,0.161685,99.893098,4.256478,2.018587,-11.008932
390,912810UA4,"May 54s, 30-Year",2024-07-15,2054-05-15,29.731507,NaN,4.625,1.0,7.642080e+10,7.428279e+09,5.191628e+09,63800.8965,4.135770,108.312500,0.565557,106.239713,4.253436,2.638344,-11.766595


In [54]:
rv_df.to_excel("rv_sheet.xlsx", index=False)

In [302]:
fig = px.scatter(
    curve_set_df,
    x="time_to_maturity",
    y=f"{quote_type}_yield",
    color="original_security_term",
    hover_data=["label"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "yield": "Yield (%)"},
    title="Filtered USTs Curve Set"
)

# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_ns(t2), mode="lines", name="Nelson-Siegel"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_nss(t2), mode="lines", name="Svensson"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_dl(t2), mode="lines", name="Diebold-Li"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_bc(t2), mode="lines", name="Björk-Christensen"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_aug_bc(t2), mode="lines", name="Augmented Björk-Christensen"
#     )
# )
fig.add_trace(
    go.Scatter(
        x=t2, y=fitted_zero_curve_dict["ql_f_nss"]["scipy_interp_func"](t2), mode="lines", name="Fitted NSS - Zero Curve"
    )
)

fig.update_layout(
    legend_title_text='Label',
    xaxis_title="Time to Maturity (Years)",
    yaxis_title=f"YTMs: {quote_type}_yield",
    width=1600,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)
fig.show()

In [280]:
pgbs = filtered_curve_set_df[["maturity_date", "int_rate", f"{quote_type}_price", "time_to_maturity"]]
display(pgbs)

calendar = ql.TARGET()
today = calendar.adjust(pydatetime_to_quantlib_date(as_of_date))
ql.Settings.instance().evaluationDate = today

bondSettlementDays = 1
bondSettlementDate = calendar.advance(
    today,
    ql.Period(bondSettlementDays, ql.Days))
frequency = ql.Annual
dc = ql.ActualActual(ql.ActualActual.ISMA)
accrualConvention = ql.ModifiedFollowing
convention = ql.ModifiedFollowing
redemption = 100.0

instruments = []
for idx, row in pgbs.iterrows():
    maturity = pydatetime_to_quantlib_date(row["maturity_date"]) 
    schedule = ql.Schedule(
        bondSettlementDate,
        maturity,
        ql.Period(frequency),
        calendar,
        accrualConvention,
        accrualConvention,
        ql.DateGeneration.Backward,
        False)
    helper = ql.FixedRateBondHelper(
            ql.QuoteHandle(ql.SimpleQuote(row[f"{quote_type}_price"])),
            bondSettlementDays,
            100.0,
            schedule,
            [row["int_rate"] / 100],
            dc,
            convention,
            redemption)

    instruments.append(helper)

params = [bondSettlementDate, instruments, dc]

fittingMethods = {
    'NelsonSiegelFitting': ql.NelsonSiegelFitting(),
    'SvenssonFitting': ql.SvenssonFitting(),
    'SimplePolynomialFitting': ql.SimplePolynomialFitting(2),
    'ExponentialSplinesFitting': ql.ExponentialSplinesFitting(),
    # 'CubicBSplinesFitting': ql.CubicBSplinesFitting([-10.0, 0, 1, 2, 5, 10.0, 20.0, 30.0, 40.0]),
}

fittedBondCurveMethods = {
    label: ql.FittedBondDiscountCurve(*params, method)
    for label, method in fittingMethods.items()
}

,maturity_date,int_rate,eod_price,time_to_maturity
53,2024-11-30,4.500,99.84375,0.260274
54,2024-12-31,4.250,99.75000,0.345205
55,2025-01-31,4.125,99.65625,0.430137
56,2025-02-28,4.625,99.93750,0.506849
57,2025-03-31,3.875,99.50000,0.591781
...,...,...,...,...
386,2053-02-15,3.625,91.00000,28.490411
387,2053-05-15,3.625,91.03125,28.734247
388,2053-08-15,4.125,99.56250,28.986301
389,2053-11-15,4.750,110.40625,29.238356


In [286]:
method = "SvenssonFitting"
curve = fittedBondCurveMethods.get(method)
curve.enableExtrapolation()

dates = [
    bondSettlementDate + ql.Period(i, ql.Days) for i in range(0, 12 * 30 * 30, 1)
]

discount_factors = [curve.discount(d) for d in dates]
ttm = [(ql.Date.to_date(d) - ql.Date.to_date(bondSettlementDate)).days / 365 for d in dates]
zero_rates = [(-np.log(df) / t) * 100 for df, t in zip(discount_factors, ttm)]
zero_rates[0] = 5.31

eq_zero_rates = []
for d in dates:
    yrs = (ql.Date.to_date(d) - ql.Date.to_date(bondSettlementDate)).days / 365.0
    zero_rate = curve.zeroRate(yrs, ql.Continuous, True)
    eq_rate = zero_rate.equivalentRate(
        dc, ql.Continuous, ql.NoFrequency, bondSettlementDate, d
    ).rate()
    eq_zero_rates.append(eq_rate * 100)
eq_zero_rates[0] = 5.31


li_zero_rates_func = scipy.interpolate.interp1d(
    ttm,
    zero_rates,
    axis=0,
    kind="linear",
    fill_value='extrapolate'
)
li_eq_zero_rates_func = scipy.interpolate.interp1d(
    ttm,
    eq_zero_rates,
    axis=0,
    kind="linear",
    fill_value='extrapolate'
)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ttm,
    y=zero_rates,
    mode='lines',
    name=f'Zero Curve {method} - QL (from DF)'
))
fig.add_trace(go.Scatter(
    x=ttm,
    y=eq_zero_rates,
    mode='lines',
    name=f'Zero Curve {method} - QL (from eq zero)'
))

t2 = np.linspace(0, 30, 1000)
fig.add_trace(go.Scatter(
    x=t2,
    y=li_zero_rates_func(t2),
    mode='lines',
    name=f'Zero Curve {method} - INTERP (from DF)'
))
fig.add_trace(go.Scatter(
    x=t2,
    y=li_eq_zero_rates_func(t2),
    mode='lines',
    name=f'Zero Curve {method} - INTERP (from eq zero)'
))

# fig.add_trace(go.Scatter(
#     x=t2,
#     y=fitted_zero_curve_func_nss(t2),
#     mode='lines',
#     name=f'Zero Curve {method} - calc'
# ))

fig.update_layout(
    title="Fitted Bond Discount Curve",
    xaxis_title="Time to Maturity (Years)",
    yaxis_title="Zero Rate (%)",
    legend_title="Curve",
    width=1600,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)

fig.show()

C:\Users\chris\AppData\Local\Temp\ipykernel_24792\4172412704.py:11: RuntimeWarning:

invalid value encountered in scalar divide

